In [ ]:
import plotly.express as px
from rindti.models import NoisyNodesClassModel, ClassificationModel, InfoGraphModel, GraphLogModel, PfamModel, BGRLModel
from rindti.models.bgrl import Encoder
from rindti.utils.data import Dataset
import pickle
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
from copy import deepcopy, copy
from rindti.utils.data import PreTrainDataset
from torch_geometric.data import DataLoader
import torch

In [ ]:
ds = PreTrainDataset("/scratch/SCRATCH_NVME/ilya/pretrain_data/merged_1000.pkl")

In [ ]:
dl = DataLoader(ds[:10000], num_workers=1, batch_size=32)

In [ ]:
def embed_batch(encoder, batch):
    batch = batch.clone()
    return encoder(batch.__dict__).detach()

In [ ]:
res = [embed_batch(encoder, x) for x in dl]    

In [ ]:
res = torch.cat(res)

In [ ]:
class TestModel(ClassificationModel):
    def embed_prot(self, x, edge_index, **kwargs):
        x = self.prot_feat_embed(x)
        x = self.prot_node_embed(x, edge_index)
        return self.prot_pool(x, edge_index, batch=None).detach().numpy().reshape(-1)
    
    def embed_drug(self, x, edge_index, **kwargs):
        x = self.drug_feat_embed(x)
        x = self.drug_node_embed(x, edge_index)
        return self.drug_pool(x, edge_index, batch=None).detach().numpy().reshape(-1)

In [ ]:
popfam = prots['fam'].value_counts().head(5)

In [ ]:
prot_sub = prots[prots['fam'].isin(popfam.index)]

In [ ]:
def embed_prot(encoder, data, **kwargs):
    data['batch'] = None
    embed = encoder(data)
    return embed.detach().numpy().reshape(-1)

In [ ]:

model = BGRLModel.load_from_checkpoint('tb_logs/bgrl/version_2/checkpoints/epoch=154-step=163989.ckpt')
model.eval()
encoder = model.student_encoder

In [ ]:
embeds = {}
for id, orig_prot in prot_sub['data'].to_dict().items():
    print(id)
    prot = copy(orig_prot)
    emb = embed_prot(encoder, prot)
    embeds[id] = emb
prot_embeds = pd.DataFrame(embeds).T

In [ ]:
prot_sub

In [ ]:
tsne = TSNE()
x = tsne.fit_transform(res)


In [ ]:
x = pd.DataFrame(index=prots_sub.index, data=x).join(prots_sub)

In [ ]:
px.scatter(x, 0, 1, opacity=0.5)